In [6]:
import os
import json
import csv
import math
import shutil
import random
import zipfile
import traceback
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
from tqdm import tqdm

from PIL import Image, ImageEnhance, ImageOps, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# =========================================================
# CONFIG (SMALL MODEL + ROBUST)
# =========================================================
CONFIG = {
    "seed": 42,
    "img_size": 224,
    "batch_size": 32,

    "epochs_phase1": 10,
    "epochs_phase2": 30,

    "learning_rate": 1e-3,
    "fine_tune_lr": 1e-5,

    "dropout_rate": 0.45,
    "label_smoothing": 0.1,
    "early_stop_patience": 10,

    "min_samples_per_class": 100,
    "max_samples_per_class": 900,

    "test_split": 0.15,
    "val_split": 0.15,

    "mixup": True,
    "mixup_alpha": 0.2,

    "use_mixed_precision": True,

    "model_name": "mobilenetv2",   # deploy-friendly
    "mobilenet_alpha": 0.75,       # 1.0 full, 0.75 smaller, 0.5 tiny
    "head_units": 128,

    "warmup_epochs": 2,
    "num_workers": tf.data.AUTOTUNE,

    "unknown_threshold": 0.55,

    "export_tflite": True,
    "export_int8": True,
    "rep_data_samples": 200,
}

# =========================================================
# YOUR KAGGLE DATASET ROOT
# =========================================================
DATA_DIR = Path("/kaggle/input/procesed-again-costal-polutant")

# ✅ Your class folders in this dataset
CLASS_SOURCES = {
    "cardboard": DATA_DIR / "cardboard",
    "glass": DATA_DIR / "glass",
    "metal": DATA_DIR / "metal",
    "paper": DATA_DIR / "paper",
    "plastic": DATA_DIR / "plastic",
    "marine_trash": DATA_DIR / "processed_marine_waste" / "marine_trash",
    "oil_spill": DATA_DIR / "processed_oilspill" / "oil_spill",
}

# Negatives (NOT trained as classes in this script)
NEGATIVE_SOURCES = {
    "clean_underwater": DATA_DIR / "processed_marine_waste" / "clean_underwater",
    "no_oil_spill": DATA_DIR / "processed_oilspill" / "no_oil_spill",
}

# =========================================================
# OUTPUTS
# =========================================================
RUN_DIR = Path("/kaggle/working/run_output")
RUN_DIR.mkdir(parents=True, exist_ok=True)

BALANCED_DIR = RUN_DIR / "balanced_dataset"
SPLIT_DIR = RUN_DIR / "splits"
TRAIN_DIR = SPLIT_DIR / "train"
VAL_DIR = SPLIT_DIR / "val"
TEST_DIR = SPLIT_DIR / "test"

REPORTS_DIR = RUN_DIR / "reports"
PLOTS_DIR = RUN_DIR / "plots"
MODELS_DIR = RUN_DIR / "models"
PREVIEW_DIR = RUN_DIR / "aug_preview"
ZIP_DIR = RUN_DIR / "zips"

for d in [REPORTS_DIR, PLOTS_DIR, MODELS_DIR, PREVIEW_DIR, ZIP_DIR]:
    d.mkdir(parents=True, exist_ok=True)

BEST_CKPT_PATH = MODELS_DIR / "best_model.weights.h5"
BEST_H5_PATH = MODELS_DIR / "best_model.h5"
SAVEDMODEL_PATH = MODELS_DIR / "saved_model"
CLASS_MAP_PATH = MODELS_DIR / "class_mapping.json"
HISTORY_CSV_PATH = REPORTS_DIR / "training_history.csv"
BALANCE_REPORT_PATH = REPORTS_DIR / "balancing_report.json"
SUMMARY_REPORT_PATH = REPORTS_DIR / "training_summary.json"
TEST_ZIP_PATH = RUN_DIR / "TEST.zip"

TFLITE_FP16_PATH = MODELS_DIR / "model_fp16.tflite"
TFLITE_INT8_PATH = MODELS_DIR / "model_int8.tflite"

# =========================================================
# Utils
# =========================================================
def set_seeds(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def status(msg: str):
    print(f"\n🟦 {msg}")

def warn(msg: str):
    print(f"🟨 {msg}")

def err(msg: str):
    print(f"🟥 {msg}")

def safe_mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def list_images(folder: Path) -> List[Path]:
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff"}
    files = []
    for fp in folder.rglob("*"):
        if fp.is_file() and fp.suffix.lower() in exts:
            files.append(fp)
    return files

def sha1_file(fp: Path, chunk_size=1024 * 1024) -> str:
    import hashlib
    h = hashlib.sha1()
    with open(fp, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def save_json(path: Path, obj: dict):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def read_image_pil(path: Path) -> Image.Image:
    im = Image.open(path)
    return im.convert("RGB")

def resize_to_square(im: Image.Image, size: int) -> Image.Image:
    return im.resize((size, size), resample=Image.BILINEAR)

def random_augment_pil(im: Image.Image) -> Image.Image:
    if random.random() < 0.5:
        im = ImageOps.mirror(im)
    if random.random() < 0.25:
        im = ImageOps.flip(im)

    angle = random.uniform(-30, 30)
    im = im.rotate(angle, resample=Image.BILINEAR, expand=False)

    if random.random() < 0.7:
        w, h = im.size
        zoom = random.uniform(0.8, 1.2)
        if zoom > 1.0:
            nw, nh = int(w / zoom), int(h / zoom)
            left = random.randint(0, max(0, w - nw))
            top = random.randint(0, max(0, h - nh))
            im = im.crop((left, top, left + nw, top + nh)).resize((w, h), Image.BILINEAR)

    if random.random() < 0.7:
        im = ImageEnhance.Brightness(im).enhance(random.uniform(0.8, 1.2))
    if random.random() < 0.7:
        im = ImageEnhance.Contrast(im).enhance(random.uniform(0.8, 1.2))

    return im

def zip_folder(folder: Path, zip_path: Path):
    if zip_path.exists():
        zip_path.unlink()
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for fp in folder.rglob("*"):
            if fp.is_file():
                zf.write(fp, arcname=str(fp.relative_to(folder)))

def enable_gpu_optimizations():
    try:
        gpus = tf.config.list_physical_devices("GPU")
        if gpus:
            status(f"GPU detected: {gpus}")
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        else:
            warn("No GPU detected. Training will be slower.")
    except Exception as e:
        warn(f"GPU optimization setup failed: {e}")

def maybe_enable_mixed_precision():
    if not CONFIG["use_mixed_precision"]:
        return
    try:
        gpus = tf.config.list_physical_devices("GPU")
        if gpus:
            from tensorflow.keras import mixed_precision
            mixed_precision.set_global_policy("mixed_float16")
            status("Mixed precision enabled (fp16).")
        else:
            warn("Mixed precision requested but no GPU found; skipping.")
    except Exception as e:
        warn(f"Mixed precision setup failed: {e}")

# =========================================================
# Data Balancer
# =========================================================
class DataBalancer:
    def __init__(self, out_dir: Path, config: dict):
        self.out_dir = out_dir
        self.config = config
        self.seed = config["seed"]

    def analyze_distribution(self) -> Dict[str, int]:
        status("Analyzing class distribution from CLASS_SOURCES...")
        class_counts = {}

        for class_name, class_path in CLASS_SOURCES.items():
            if not class_path.exists():
                warn(f"Missing class folder: {class_name} -> {class_path}")
                class_counts[class_name] = 0
                continue
            class_counts[class_name] = len(list_images(class_path))

        total = sum(class_counts.values())
        print("Class distribution (BEFORE):")
        for k, v in class_counts.items():
            print(f"  - {k}: {v}")
        print(f"Total images: {total}")
        return class_counts

    def apply_augmentation(self, src_paths: List[Path], needed: int, class_name: str, dst_class_dir: Path):
        safe_mkdir(dst_class_dir)
        random.shuffle(src_paths)
        status(f"Augmenting '{class_name}' with {needed} new samples (offline)...")

        for i in tqdm(range(needed), desc=f"Augmenting {class_name}"):
            try:
                base = src_paths[i % len(src_paths)]
                im = read_image_pil(base)
                im = random_augment_pil(im)
                im = resize_to_square(im, self.config["img_size"])
                out_path = dst_class_dir / f"aug_{class_name}_{i:07d}.jpg"
                im.save(out_path, format="JPEG", quality=90, optimize=True)
            except Exception:
                continue

    def balance_classes(self) -> Dict[str, Dict[str, int]]:
        set_seeds(self.seed)

        # wipe old balanced dir
        if self.out_dir.exists():
            shutil.rmtree(self.out_dir, ignore_errors=True)
        safe_mkdir(self.out_dir)

        before = self.analyze_distribution()
        counts = [v for v in before.values() if v > 0]
        if not counts:
            raise ValueError("No valid images found in any class folders.")

        min_count = min(counts)

        target_count = max(self.config["min_samples_per_class"], min_count)
        target_count = min(target_count, self.config["max_samples_per_class"])

        status(f"Balancing target per class: {target_count} images")

        seen_hashes = set()
        report = {"before": before, "after": {}, "target_count": target_count}

        for class_name in CLASS_SOURCES.keys():
            src_class_dir = CLASS_SOURCES[class_name]
            dst_class_dir = self.out_dir / class_name
            safe_mkdir(dst_class_dir)

            if not src_class_dir.exists():
                warn(f"Skipping missing class folder: {src_class_dir}")
                report["after"][class_name] = 0
                continue

            all_imgs = list_images(src_class_dir)
            random.shuffle(all_imgs)

            # Global dedupe by hash
            deduped = []
            for fp in all_imgs:
                try:
                    h = sha1_file(fp)
                except Exception:
                    continue
                if h in seen_hashes:
                    continue
                seen_hashes.add(h)
                deduped.append(fp)

            if len(deduped) == 0:
                warn(f"Class '{class_name}' has 0 valid images after dedupe.")
                report["after"][class_name] = 0
                continue

            if len(deduped) > target_count:
                keep = deduped[:target_count]
                status(f"'{class_name}': {len(deduped)} -> keeping {target_count} (random deletion)")
                for i, fp in enumerate(tqdm(keep, desc=f"Copying {class_name}")):
                    try:
                        im = read_image_pil(fp)
                        im = resize_to_square(im, self.config["img_size"])
                        out_path = dst_class_dir / f"{class_name}_{i:07d}.jpg"
                        im.save(out_path, format="JPEG", quality=90, optimize=True)
                    except Exception:
                        continue
            else:
                status(f"'{class_name}': {len(deduped)} -> boosting to {target_count} (offline augmentation)")
                for i, fp in enumerate(tqdm(deduped, desc=f"Copying {class_name}")):
                    try:
                        im = read_image_pil(fp)
                        im = resize_to_square(im, self.config["img_size"])
                        out_path = dst_class_dir / f"{class_name}_{i:07d}.jpg"
                        im.save(out_path, format="JPEG", quality=90, optimize=True)
                    except Exception:
                        continue

                current = len(list(dst_class_dir.glob("*.jpg")))
                needed = max(0, target_count - current)
                if needed > 0:
                    self.apply_augmentation(deduped, needed, class_name, dst_class_dir)

            report["after"][class_name] = len(list(dst_class_dir.glob("*.jpg")))

        save_json(BALANCE_REPORT_PATH, report)
        status(f"Balancing report saved to: {BALANCE_REPORT_PATH}")

        print("Class distribution (AFTER):")
        for k, v in report["after"].items():
            print(f"  - {k}: {v}")

        return report

# =========================================================
# Trainer
# =========================================================
class WarmupThenConstant(tf.keras.callbacks.Callback):
    def __init__(self, base_lr: float, warmup_epochs: int):
        super().__init__()
        self.base_lr = float(base_lr)
        self.warmup_epochs = int(warmup_epochs)

    def on_epoch_begin(self, epoch, logs=None):
        if epoch < self.warmup_epochs:
            lr = self.base_lr * (epoch + 1) / self.warmup_epochs
        else:
            lr = self.base_lr

        opt = self.model.optimizer
        # Keras-safe LR set (handles Variable / schedule / weird wrappers)
        try:
            if hasattr(opt, "learning_rate"):
                if hasattr(opt.learning_rate, "assign"):
                    opt.learning_rate.assign(lr)
                else:
                    # some optimizers keep lr inside _learning_rate
                    if hasattr(opt, "_learning_rate") and hasattr(opt._learning_rate, "assign"):
                        opt._learning_rate.assign(lr)
                    else:
                        # last resort: just replace attribute
                        opt.learning_rate = lr
        except Exception as e:
            print(f"🟨 Warmup LR set failed, skipping. Reason: {e}")


class AntiOverfitTrainer:
    def __init__(self, config: dict):
        self.config = config
        self.img_size = config["img_size"]
        self.batch_size = config["batch_size"]
        self.model = None
        self.class_names = None

    def _build_augmenter(self):
        return keras.Sequential([
            layers.RandomFlip(mode="horizontal_and_vertical"),
            layers.RandomRotation(factor=30/360),
            layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),
            layers.RandomContrast(factor=0.2),
            layers.RandomBrightness(factor=0.2),
        ], name="augmenter")

    def _random_crop_resize(self, img, lbl):
        size = self.img_size
        bigger = int(size * 1.15)
        img = tf.image.resize(img, (bigger, bigger))
        img = tf.image.random_crop(img, size=[size, size, 3])
        return img, lbl

    def _preprocess_input(self, x):
        return tf.keras.applications.mobilenet_v2.preprocess_input(x)

    def _base_model(self):
        input_shape = (self.img_size, self.img_size, 3)
        base = tf.keras.applications.MobileNetV2(
            include_top=False,
            weights="imagenet",
            input_shape=input_shape,
            alpha=self.config.get("mobilenet_alpha", 1.0),
        )
        return base

    def build_model(self, class_names: List[str]):
        status("Building SMALL deployable model (MobileNetV2)...")
        self.class_names = class_names
        num_classes = len(class_names)

        inputs = keras.Input(shape=(self.img_size, self.img_size, 3), name="image")
        base = self._base_model()
        base.trainable = False

        x = self._preprocess_input(inputs)
        x = base(x, training=False)

        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(
            self.config["head_units"],
            kernel_regularizer=regularizers.l2(1e-4)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.Dropout(self.config["dropout_rate"])(x)

        outputs = layers.Dense(
            num_classes,
            activation="softmax",
            dtype="float32",
            name="pred"
        )(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs, name="small_classifier")
        status(f"Model built. Classes={num_classes}, alpha={self.config.get('mobilenet_alpha')}, head={self.config['head_units']}")
        return self.model

    def compile_with_regularization(self, lr: float):
        status(f"Compiling model (lr={lr})...")
        try:
            opt = tf.keras.optimizers.AdamW(
                learning_rate=lr,
                weight_decay=1e-4,
                clipnorm=1.0
            )
        except Exception:
            opt = tf.keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)

        loss = tf.keras.losses.CategoricalCrossentropy(
            label_smoothing=self.config["label_smoothing"]
        )

        self.model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

    def _mixup_batch(self, x, y):
    # Fix mixed precision dtype mismatch:
    # x can be float16 under mixed precision, y is float32 → crash
        x = tf.cast(x, tf.float32)
        y = tf.cast(y, tf.float32)
    
        alpha = self.config["mixup_alpha"]
        if alpha <= 0:
            return x, y
    
        lam = tf.cast(tf.random.gamma([], alpha, beta=1.0) /
                      (tf.random.gamma([], alpha, beta=1.0) + tf.random.gamma([], alpha, beta=1.0)),
                      tf.float32)
    
        bs = tf.shape(x)[0]
        idx = tf.random.shuffle(tf.range(bs))
        x2 = tf.gather(x, idx)
        y2 = tf.gather(y, idx)
    
        x_mix = lam * x + (1.0 - lam) * x2
        y_mix = lam * y + (1.0 - lam) * y2
        return x_mix, y_mix


    def _make_dataset(self, files: List[str], labels_oh: np.ndarray, training: bool):
        ds = tf.data.Dataset.from_tensor_slices((files, labels_oh))

        def _load(path, y):
            img = tf.io.read_file(path)
            img = tf.image.decode_image(img, channels=3, expand_animations=False)
            img = tf.image.resize(img, (self.img_size, self.img_size))
            img = tf.cast(img, tf.float32)
            return img, y

        ds = ds.map(_load, num_parallel_calls=self.config["num_workers"])

        if training:
            aug = self._build_augmenter()
            ds = ds.map(self._random_crop_resize, num_parallel_calls=self.config["num_workers"])
            ds = ds.map(lambda x, y: (aug(x, training=True), y),
                        num_parallel_calls=self.config["num_workers"])

            ds = ds.shuffle(2048, seed=self.config["seed"], reshuffle_each_iteration=True)

            if self.config["mixup"]:
                ds = ds.batch(self.batch_size, drop_remainder=True)
                ds = ds.map(self._mixup_batch, num_parallel_calls=self.config["num_workers"])
            else:
                ds = ds.batch(self.batch_size)
        else:
            ds = ds.batch(self.batch_size)

        return ds.prefetch(self.config["num_workers"])

    def train_with_callbacks(self, train_ds, val_ds, phase: str):
        status(f"Training {phase}...")

        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=self.config["early_stop_patience"],
                restore_best_weights=True
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                filepath=str(BEST_CKPT_PATH),
                monitor="val_loss",
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            ),
        ]
        if phase == "phase1" and self.config["warmup_epochs"] > 0:
            callbacks.append(WarmupThenConstant(self.config["learning_rate"], self.config["warmup_epochs"]))

        epochs = self.config["epochs_phase1"] if phase == "phase1" else self.config["epochs_phase2"]

        hist = self.model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=callbacks
        )
        return hist

    def evaluate(self, test_ds, y_true_idx: np.ndarray, class_names: List[str]):
        status("Evaluating on test set...")
        if BEST_CKPT_PATH.exists():
            self.model.load_weights(str(BEST_CKPT_PATH))
            status("Loaded best checkpoint for evaluation.")

        y_prob = self.model.predict(test_ds, verbose=1)
        y_pred = np.argmax(y_prob, axis=1)

        cm = confusion_matrix(y_true_idx, y_pred)
        report = classification_report(
            y_true_idx, y_pred,
            target_names=class_names,
            output_dict=True
        )

        per_class_acc = {}
        for i, name in enumerate(class_names):
            denom = cm[i].sum()
            per_class_acc[name] = float(cm[i, i] / denom) if denom > 0 else 0.0

        thr = self.config["unknown_threshold"]
        maxp = np.max(y_prob, axis=1)
        unknown_rate = float(np.mean(maxp < thr))

        return {
            "confusion_matrix": cm.tolist(),
            "classification_report": report,
            "per_class_accuracy": per_class_acc,
            "unknown_threshold": thr,
            "unknown_rate_on_test": unknown_rate
        }

# =========================================================
# Split helpers
# =========================================================
def build_file_label_lists(root: Path) -> Tuple[List[str], List[str]]:
    classes = sorted([d.name for d in root.iterdir() if d.is_dir()])
    files, labels = [], []
    for c in classes:
        imgs = list_images(root / c)
        for fp in imgs:
            files.append(str(fp))
            labels.append(c)
    return files, labels

def export_split_to_folders(split_name: str, files: List[str], labels: List[str], out_root: Path):
    status(f"Exporting split '{split_name}'...")
    if out_root.exists():
        shutil.rmtree(out_root, ignore_errors=True)
    safe_mkdir(out_root)

    for fp, cls in tqdm(list(zip(files, labels)), desc=f"Copying {split_name}"):
        try:
            src = Path(fp)
            dst_dir = out_root / cls
            safe_mkdir(dst_dir)
            shutil.copy2(src, dst_dir / src.name)
        except Exception:
            continue

def make_test_zip(test_dir: Path, zip_path: Path):
    status(f"Creating TEST.zip from: {test_dir}")
    zip_folder(test_dir, zip_path)
    status(f"TEST.zip created: {zip_path} ({zip_path.stat().st_size / (1024*1024):.2f} MB)")

def one_hot(labels: List[str], class_to_idx: Dict[str, int]) -> np.ndarray:
    y = np.zeros((len(labels), len(class_to_idx)), dtype=np.float32)
    for i, c in enumerate(labels):
        y[i, class_to_idx[c]] = 1.0
    return y

# =========================================================
# TFLite export
# =========================================================
def export_tflite_fp16(model: tf.keras.Model):
    status("Exporting TFLite FP16...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()
    TFLITE_FP16_PATH.write_bytes(tflite_model)
    status(f"Saved: {TFLITE_FP16_PATH} ({TFLITE_FP16_PATH.stat().st_size/(1024*1024):.2f} MB)")

def export_tflite_int8(model: tf.keras.Model, rep_paths: List[str], img_size: int):
    status("Exporting TFLite INT8 (smallest)...")

    def representative_dataset():
        for p in rep_paths:
            try:
                img = tf.io.read_file(p)
                img = tf.image.decode_image(img, channels=3, expand_animations=False)
                img = tf.image.resize(img, (img_size, img_size))
                img = tf.cast(img, tf.float32)
                img = tf.expand_dims(img, axis=0)
                yield [img]
            except Exception:
                continue

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8

    tflite_model = converter.convert()
    TFLITE_INT8_PATH.write_bytes(tflite_model)
    status(f"Saved: {TFLITE_INT8_PATH} ({TFLITE_INT8_PATH.stat().st_size/(1024*1024):.2f} MB)")

# =========================================================
# main
# =========================================================
def main():
    try:
        set_seeds(CONFIG["seed"])
        enable_gpu_optimizations()
        maybe_enable_mixed_precision()

        status("Starting training pipeline on your Kaggle dataset structure...")
        status(f"DATA_DIR: {DATA_DIR}")
        if not DATA_DIR.exists():
            raise FileNotFoundError(f"DATA_DIR not found: {DATA_DIR}")

        # Quick folder existence check
        status("Checking class source folders...")
        for k, v in CLASS_SOURCES.items():
            print(f"  {k:12s} -> exists={v.exists()} path={v}")

        # 1) Balance (creates BALANCED_DIR/class_name/*.jpg)
        balancer = DataBalancer(BALANCED_DIR, CONFIG)
        balance_report = balancer.balance_classes()

        # 2) Stratified splits
        status("Creating stratified splits from balanced dataset...")
        files, labels = build_file_label_lists(BALANCED_DIR)
        classes = sorted(list(set(labels)))
        class_to_idx = {c: i for i, c in enumerate(classes)}
        save_json(CLASS_MAP_PATH, {"classes": classes, "class_to_idx": class_to_idx})
        status(f"Saved class mapping: {CLASS_MAP_PATH}")

        X_trainval, X_test, y_trainval, y_test = train_test_split(
            files, labels,
            test_size=CONFIG["test_split"],
            random_state=CONFIG["seed"],
            stratify=labels
        )

        val_relative = CONFIG["val_split"] / (1.0 - CONFIG["test_split"])
        X_train, X_val, y_train, y_val = train_test_split(
            X_trainval, y_trainval,
            test_size=val_relative,
            random_state=CONFIG["seed"],
            stratify=y_trainval
        )

        status(f"Split sizes: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")

        export_split_to_folders("train", X_train, y_train, TRAIN_DIR)
        export_split_to_folders("val", X_val, y_val, VAL_DIR)
        export_split_to_folders("test", X_test, y_test, TEST_DIR)

        make_test_zip(TEST_DIR, TEST_ZIP_PATH)

        # rebuild split lists (ensures no leakage)
        train_files, train_labels = build_file_label_lists(TRAIN_DIR)
        val_files, val_labels = build_file_label_lists(VAL_DIR)
        test_files, test_labels = build_file_label_lists(TEST_DIR)

        y_train_oh = one_hot(train_labels, class_to_idx)
        y_val_oh = one_hot(val_labels, class_to_idx)
        y_test_oh = one_hot(test_labels, class_to_idx)

        # 3) Build + Train
        trainer = AntiOverfitTrainer(CONFIG)
        model = trainer.build_model(classes)

        train_ds = trainer._make_dataset(train_files, y_train_oh, training=True)
        val_ds = trainer._make_dataset(val_files, y_val_oh, training=False)
        test_ds = trainer._make_dataset(test_files, y_test_oh, training=False)

        # Resume support
        trainer.compile_with_regularization(CONFIG["learning_rate"])
        if BEST_CKPT_PATH.exists():
            warn("Checkpoint found. Loading weights to resume.")
            model.load_weights(str(BEST_CKPT_PATH))

        hist_all = {"loss": [], "accuracy": [], "val_loss": [], "val_accuracy": []}

        # Phase 1
        h1 = trainer.train_with_callbacks(train_ds, val_ds, "phase1")
        for k in hist_all:
            if k in h1.history:
                hist_all[k].extend(h1.history[k])

        # Phase 2: unfreeze top 30%
        status("Fine-tuning: unfreezing top 30% layers...")
        base_model = None
        for layer in model.layers:
            if isinstance(layer, tf.keras.Model) and "mobilenet" in layer.name.lower():
                base_model = layer
                break

        if base_model is not None:
            base_model.trainable = True
            n_layers = len(base_model.layers)
            cut = int(n_layers * 0.7)
            for i, layer in enumerate(base_model.layers):
                layer.trainable = (i >= cut)
            status(f"Base layers: {n_layers}, trainable: {n_layers - cut}, frozen: {cut}")
        else:
            warn("Could not find MobileNet base model block; fine-tune may be limited.")

        trainer.compile_with_regularization(CONFIG["fine_tune_lr"])
        if BEST_CKPT_PATH.exists():
            model.load_weights(str(BEST_CKPT_PATH))
            status("Loaded best checkpoint before fine-tune.")

        h2 = trainer.train_with_callbacks(train_ds, val_ds, "phase2")
        for k in hist_all:
            if k in h2.history:
                hist_all[k].extend(h2.history[k])

        # Save history CSV
        status("Saving history CSV...")
        with open(HISTORY_CSV_PATH, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(list(hist_all.keys()))
            for i in range(len(hist_all["loss"])):
                writer.writerow([hist_all[k][i] for k in hist_all])

        # Overfit indicator
        gaps = [float(a - b) for a, b in zip(hist_all["accuracy"], hist_all["val_accuracy"])]
        overfit_summary = {
            "max_train_val_acc_gap": float(np.max(gaps)) if gaps else None,
            "mean_train_val_acc_gap": float(np.mean(gaps)) if gaps else None
        }
        status(f"Overfit gap summary: {overfit_summary}")

        # Evaluate
        y_test_idx = np.array([class_to_idx[c] for c in test_labels], dtype=np.int32)
        eval_out = trainer.evaluate(test_ds, y_test_idx, classes)

        # Save model
        status("Saving best model (.h5 + SavedModel)...")
        if BEST_CKPT_PATH.exists():
            model.load_weights(str(BEST_CKPT_PATH))

        model.save(str(BEST_H5_PATH), include_optimizer=False)
        if SAVEDMODEL_PATH.exists():
            shutil.rmtree(SAVEDMODEL_PATH, ignore_errors=True)
        model.save(str(SAVEDMODEL_PATH))

        status(f"Saved .h5: {BEST_H5_PATH} ({BEST_H5_PATH.stat().st_size/(1024*1024):.2f} MB)")

        # TFLite exports
        if CONFIG["export_tflite"]:
            export_tflite_fp16(model)
            if CONFIG["export_int8"]:
                rep_n = min(CONFIG["rep_data_samples"], len(train_files))
                rep_paths = random.sample(train_files, rep_n) if rep_n > 0 else train_files[:rep_n]
                export_tflite_int8(model, rep_paths, CONFIG["img_size"])

        # Summary report
        summary = {
            "config": CONFIG,
            "classes": classes,
            "class_mapping": str(CLASS_MAP_PATH),
            "balance_report": str(BALANCE_REPORT_PATH),
            "history_csv": str(HISTORY_CSV_PATH),
            "test_zip": str(TEST_ZIP_PATH),
            "models": {
                "best_h5": str(BEST_H5_PATH),
                "saved_model": str(SAVEDMODEL_PATH),
                "best_weights": str(BEST_CKPT_PATH),
                "tflite_fp16": str(TFLITE_FP16_PATH) if TFLITE_FP16_PATH.exists() else None,
                "tflite_int8": str(TFLITE_INT8_PATH) if TFLITE_INT8_PATH.exists() else None,
            },
            "overfit_indicators": overfit_summary,
            "evaluation": eval_out
        }
        save_json(SUMMARY_REPORT_PATH, summary)
        status(f"Saved summary: {SUMMARY_REPORT_PATH}")

        # Zip models for download
        models_zip = ZIP_DIR / "models_bundle.zip"
        status("Zipping MODELS_DIR for easy download...")
        zip_folder(MODELS_DIR, models_zip)
        status(f"Download: {models_zip} ({models_zip.stat().st_size/(1024*1024):.2f} MB)")

        status("✅ DONE. Go to Kaggle Output panel and download run_output/zips/models_bundle.zip + TEST.zip")

    except Exception as e:
        err("Pipeline crashed 😭")
        print("Error:", e)
        print(traceback.format_exc())

if __name__ == "__main__":
    main()



🟦 GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

🟦 Mixed precision enabled (fp16).

🟦 Starting training pipeline on your Kaggle dataset structure...

🟦 DATA_DIR: /kaggle/input/procesed-again-costal-polutant

🟦 Checking class source folders...
  cardboard    -> exists=True path=/kaggle/input/procesed-again-costal-polutant/cardboard
  glass        -> exists=True path=/kaggle/input/procesed-again-costal-polutant/glass
  metal        -> exists=True path=/kaggle/input/procesed-again-costal-polutant/metal
  paper        -> exists=True path=/kaggle/input/procesed-again-costal-polutant/paper
  plastic      -> exists=True path=/kaggle/input/procesed-again-costal-polutant/plastic
  marine_trash -> exists=True path=/kaggle/input/procesed-again-costal-polutant/processed_marine_waste/marine_trash
  oil_spill    -> exists=True path=/kaggle/input/procesed-again-costal-polutant/processed_oilspill/oil_

Copying cardboard: 100%|██████████| 768/768 [00:01<00:00, 618.47it/s]



🟦 'glass': 1998 -> keeping 768 (random deletion)


Copying glass: 100%|██████████| 768/768 [00:01<00:00, 630.27it/s]



🟦 'metal': 993 -> keeping 768 (random deletion)


...


🟦 Creating TEST.zip from: /kaggle/working/run_output/splits/test

🟦 TEST.zip created: /kaggle/working/run_output/TEST.zip (8.72 MB)

🟦 Building SMALL deployable model (MobileNetV2)...

🟦 Model built. Classes=7, alpha=0.75, head=128

🟦 Compiling model (lr=0.001)...

🟦 Training phase1...
Epoch 1/10


I0000 00:00:1770139353.942102     128 service.cc:152] XLA service 0x786edc216fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770139353.942146     128 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1770139353.942151     128 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1770139355.818711     128 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1770139372.305169     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.3571 - loss: nan
Epoch 1: val_loss did not improve from inf
117/117 ━━━━━━━━━━━━━━━━━━━━ 85s 330ms/step - accuracy: 0.3580 - loss: nan - val_accuracy: 0.1425 - val_loss: nan - learning_rate: 5.0000e-04
Epoch 2/10
115/117 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.5606 - loss: nan
Epoch 2: val_loss did not improve from inf
117/117 ━━━━━━━━━━━━━━━━━━━━ 30s 127ms/step - accuracy: 0.5616 - loss: nan - val_accuracy: 0.1425 - val_loss: nan - learning_rate: 0.0010
Epoch 3/10
116/117 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.6077 - loss: nan
Epoch 3: val_loss did not improve from inf
117/117 ━━━━━━━━━━━━━━━━━━━━ 30s 128ms/step - accuracy: 0.6082 - loss: nan - val_accuracy: 0.1425 - val_loss: nan - learning_rate: 0.0010
Epoch 4/10
115/117 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.6206 - loss: nan
Epoch 4: val_loss did not improve from inf
117/117 ━━━━━━━━━━━━━━━━━━━━ 30s 127ms/step - accuracy: 0.6215 - loss: nan - val_

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🟦 Saving best model (.h5 + SavedModel)...
🟥 Pipeline crashed 😭
Error: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/kaggle/working/run_output/models/saved_model.
Traceback (most recent call last):
  File "/tmp/ipykernel_55/1363905620.py", line 827, in main
    model.save(str(SAVEDMODEL_PATH))
  File "/usr/local/lib/python3.12/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_api.py", line 114, in save_model
    raise ValueError(
ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to exp

In [8]:
import os, json, csv, shutil, zipfile, random, math
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageOps, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt

# =========================
# CONFIG (minimal + stable)
# =========================
CONFIG = {
    "seed": 42,
    "img_size": 224,
    "batch_size": 32,

    "epochs": 20,
    "fine_tune_epochs": 8,

    "learning_rate": 3e-4,
    "fine_tune_lr": 1e-5,

    "dropout": 0.35,
    "l2": 1e-4,
    "label_smoothing": 0.05,

    "val_split": 0.15,
    "test_split": 0.15,

    "early_stop_patience": 6,
    "reduce_lr_patience": 3,

    # ===== Balancing rules =====
    "balance_target": "median",       # "median" or "min" or "fixed"
    "fixed_target": 800,              # used only if balance_target="fixed"
    "max_target_cap": 1200,           # don’t inflate dataset too much
    "allow_deletion": True,           # delete ONLY if class >> target * delete_factor
    "delete_factor": 1.8,             # if class_count > target*1.8 -> downsample to target
    "max_delete_fraction": 0.6,       # never delete more than 60% of a class

    # model size
    "mobilenet_alpha": 0.75,          # small model
}

# =========================
# DATASET PATH (your Kaggle structure)
# =========================
DATA_DIR = Path("/kaggle/input/procesed-again-costal-polutant")

CLASS_SOURCES = {
    "cardboard": DATA_DIR / "cardboard",
    "glass": DATA_DIR / "glass",
    "metal": DATA_DIR / "metal",
    "paper": DATA_DIR / "paper",
    "plastic": DATA_DIR / "plastic",
    "marine_trash": DATA_DIR / "processed_marine_waste" / "marine_trash",
    "oil_spill": DATA_DIR / "processed_oilspill" / "oil_spill",
}

# =========================
# OUTPUTS
# =========================
RUN_DIR = Path("/kaggle/working/run_min_balance")
BALANCED_DIR = RUN_DIR / "balanced_dataset"
SPLITS_DIR = RUN_DIR / "splits"
TRAIN_DIR = SPLITS_DIR / "train"
VAL_DIR = SPLITS_DIR / "val"
TEST_DIR = SPLITS_DIR / "test"

MODELS_DIR = RUN_DIR / "models"
PLOTS_DIR = RUN_DIR / "plots"
REPORTS_DIR = RUN_DIR / "reports"

for d in [RUN_DIR, BALANCED_DIR, SPLITS_DIR, MODELS_DIR, PLOTS_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

BEST_WEIGHTS = MODELS_DIR / "best.weights.h5"
MODEL_H5 = MODELS_DIR / "model_best.h5"
MODEL_KERAS = MODELS_DIR / "model_best.keras"
SAVEDMODEL_DIR = MODELS_DIR / "saved_model"
CLASS_MAP = MODELS_DIR / "class_mapping.json"
TEST_ZIP = RUN_DIR / "TEST.zip"

COUNTS_JSON = REPORTS_DIR / "class_counts.json"
BALANCE_REPORT_JSON = REPORTS_DIR / "balance_report.json"
HISTORY_CSV = REPORTS_DIR / "history.csv"
CLASS_REPORT_JSON = REPORTS_DIR / "classification_report.json"
CONFUSION_JSON = REPORTS_DIR / "confusion_matrix.json"


# =========================
# Utils
# =========================
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def status(msg): print(f"\n🟦 {msg}")
def warn(msg): print(f"🟨 {msg}")

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    out = []
    for fp in folder.rglob("*"):
        if fp.is_file() and fp.suffix.lower() in exts:
            out.append(fp)
    return out

def save_json(path: Path, obj: dict):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def zip_folder(folder: Path, zip_path: Path):
    if zip_path.exists():
        zip_path.unlink()
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for fp in folder.rglob("*"):
            if fp.is_file():
                zf.write(fp, arcname=str(fp.relative_to(folder)))

def enable_gpu_memory_growth():
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        status(f"GPU detected: {gpus}")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        warn("No GPU detected.")

def resize_to_square(im: Image.Image, size: int):
    return im.resize((size, size), Image.BILINEAR)

def random_augment_pil(im: Image.Image):
    # Minimal but effective augmentation
    if random.random() < 0.5:
        im = ImageOps.mirror(im)
    if random.random() < 0.2:
        im = ImageOps.flip(im)

    angle = random.uniform(-20, 20)
    im = im.rotate(angle, resample=Image.BILINEAR, expand=False)

    if random.random() < 0.6:
        im = ImageEnhance.Brightness(im).enhance(random.uniform(0.85, 1.15))
    if random.random() < 0.6:
        im = ImageEnhance.Contrast(im).enhance(random.uniform(0.85, 1.15))
    return im


# =========================
# Step 1: Count images
# =========================
def count_images_per_class():
    status("Counting images per class...")
    counts = {}
    for cls, folder in CLASS_SOURCES.items():
        if not folder.exists():
            warn(f"Missing: {folder}")
            counts[cls] = 0
            continue
        counts[cls] = len(list_images(folder))

    total = sum(counts.values())
    for cls in sorted(counts.keys()):
        print(f"  - {cls:12s}: {counts[cls]}")
    print(f"Total images: {total}")

    save_json(COUNTS_JSON, counts)
    status(f"Saved counts: {COUNTS_JSON}")
    return counts


# =========================
# Step 2: Minimal balance (augment only, delete only if extreme)
# =========================
def choose_target(counts: Dict[str, int]) -> int:
    valid = [v for v in counts.values() if v > 0]
    if not valid:
        raise ValueError("No images found in dataset.")
    if CONFIG["balance_target"] == "min":
        t = min(valid)
    elif CONFIG["balance_target"] == "fixed":
        t = int(CONFIG["fixed_target"])
    else:
        t = int(np.median(valid))
    t = min(t, CONFIG["max_target_cap"])
    return max(t, 10)

def build_balanced_dataset(counts: Dict[str, int]):
    status("Building balanced dataset (augment minority, delete only if necessary)...")

    # clean old
    if BALANCED_DIR.exists():
        shutil.rmtree(BALANCED_DIR, ignore_errors=True)
    BALANCED_DIR.mkdir(parents=True, exist_ok=True)

    target = choose_target(counts)
    status(f"Balance target per class: {target}")

    report = {"target": target, "before": counts, "after": {}, "actions": {}}

    for cls, folder in CLASS_SOURCES.items():
        out_cls = BALANCED_DIR / cls
        out_cls.mkdir(parents=True, exist_ok=True)

        imgs = list_images(folder) if folder.exists() else []
        if len(imgs) == 0:
            warn(f"Class {cls} has 0 images, skipping.")
            report["after"][cls] = 0
            report["actions"][cls] = {"copied": 0, "augmented": 0, "deleted": 0}
            continue

        random.shuffle(imgs)

        # Decide if we need to delete
        copied_list = imgs
        deleted = 0
        if CONFIG["allow_deletion"] and len(imgs) > int(target * CONFIG["delete_factor"]):
            # don't delete more than max_delete_fraction
            max_keep = int(len(imgs) * (1 - CONFIG["max_delete_fraction"]))
            keep = max(target, max_keep)
            keep = min(keep, len(imgs))
            copied_list = imgs[:keep]
            deleted = len(imgs) - keep
            status(f"Downsampling '{cls}': {len(imgs)} -> {keep} (deleted={deleted})")

        # Copy originals (resized to jpg)
        copied = 0
        for i, fp in enumerate(tqdm(copied_list, desc=f"Copy {cls}")):
            try:
                im = Image.open(fp).convert("RGB")
                im = resize_to_square(im, CONFIG["img_size"])
                im.save(out_cls / f"{cls}_{i:07d}.jpg", quality=90, optimize=True)
                copied += 1
            except Exception:
                continue

        # Augment only if below target
        augmented = 0
        if copied < target:
            need = target - copied
            status(f"Augmenting '{cls}' by {need} to reach {target}...")
            # Use the copied images as base for augmentation
            base_paths = list(out_cls.glob("*.jpg"))
            if len(base_paths) == 0:
                base_paths = [Path(p) for p in copied_list if p.exists()]

            for j in tqdm(range(need), desc=f"Aug {cls}"):
                try:
                    base = random.choice(base_paths)
                    im = Image.open(base).convert("RGB")
                    im = random_augment_pil(im)
                    im = resize_to_square(im, CONFIG["img_size"])
                    im.save(out_cls / f"aug_{cls}_{j:07d}.jpg", quality=88, optimize=True)
                    augmented += 1
                except Exception:
                    continue

        after_count = len(list(out_cls.glob("*.jpg")))
        report["after"][cls] = after_count
        report["actions"][cls] = {"copied": copied, "augmented": augmented, "deleted": deleted}

    save_json(BALANCE_REPORT_JSON, report)
    status(f"Saved balance report: {BALANCE_REPORT_JSON}")
    return report


# =========================
# Split helpers
# =========================
def build_file_label_lists_from_folder(root: Path, classes: List[str]):
    files, labels = [], []
    for cls in classes:
        folder = root / cls
        if not folder.exists():
            continue
        imgs = list_images(folder)
        files.extend([str(p) for p in imgs])
        labels.extend([cls] * len(imgs))
    return files, labels

def export_split(split_name: str, files: list, labels: list, out_dir: Path):
    status(f"Exporting {split_name} split to: {out_dir}")
    if out_dir.exists():
        shutil.rmtree(out_dir, ignore_errors=True)
    out_dir.mkdir(parents=True, exist_ok=True)

    for fp, cls in tqdm(list(zip(files, labels)), desc=f"Copying {split_name}"):
        src = Path(fp)
        dst = out_dir / cls
        dst.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst / src.name)


# =========================
# tf.data
# =========================
def make_dataset(files, labels_idx, num_classes, training: bool):
    img_size = CONFIG["img_size"]
    batch = CONFIG["batch_size"]

    y = tf.one_hot(labels_idx, depth=num_classes)
    ds = tf.data.Dataset.from_tensor_slices((files, y))

    def _load(path, y):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (img_size, img_size))
        img = tf.cast(img, tf.float32)
        return img, y

    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.08),
            layers.RandomZoom(0.15),
            layers.RandomContrast(0.15),
        ])
        ds = ds.shuffle(2048, seed=CONFIG["seed"], reshuffle_each_iteration=True)
        ds = ds.map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds


# =========================
# Model
# =========================
def build_model(num_classes: int):
    status("Building MobileNetV2 (small + stable)...")
    inputs = keras.Input(shape=(CONFIG["img_size"], CONFIG["img_size"], 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)

    base = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(CONFIG["img_size"], CONFIG["img_size"], 3),
        alpha=CONFIG["mobilenet_alpha"],
    )
    base.trainable = False

    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, kernel_regularizer=regularizers.l2(CONFIG["l2"]))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(CONFIG["dropout"])(x)

    outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)
    model = keras.Model(inputs, outputs)
    return model, base

def compile_model(model, lr):
    opt = keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)
    loss = keras.losses.CategoricalCrossentropy(label_smoothing=CONFIG["label_smoothing"])
    model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])


# =========================
# Plots
# =========================
def plot_history(history, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)

    plt.figure()
    plt.plot(history["accuracy"], label="train_acc")
    plt.plot(history["val_accuracy"], label="val_acc")
    plt.legend(); plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Accuracy")
    plt.savefig(out_dir / "accuracy.png", dpi=150)
    plt.close()

    plt.figure()
    plt.plot(history["loss"], label="train_loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend(); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Loss")
    plt.savefig(out_dir / "loss.png", dpi=150)
    plt.close()

def plot_confusion_matrix(cm, class_names, out_path: Path):
    plt.figure(figsize=(9, 9))
    plt.imshow(cm)
    plt.title("Confusion Matrix")
    plt.xlabel("Pred"); plt.ylabel("True")
    plt.colorbar()
    plt.xticks(range(len(class_names)), class_names, rotation=45, ha="right")
    plt.yticks(range(len(class_names)), class_names)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

def plot_per_class_acc(cm, class_names, out_path: Path):
    per_acc = []
    for i in range(len(class_names)):
        denom = cm[i].sum()
        per_acc.append(cm[i, i] / denom if denom > 0 else 0.0)

    plt.figure(figsize=(10, 4))
    plt.bar(class_names, per_acc)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Accuracy")
    plt.title("Per-class Accuracy")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()


# =========================
# MAIN
# =========================
def main():
    set_seeds(CONFIG["seed"])
    enable_gpu_memory_growth()

    counts = count_images_per_class()
    build_balanced_dataset(counts)

    classes = sorted(list(CLASS_SOURCES.keys()))
    class_to_idx = {c: i for i, c in enumerate(classes)}
    save_json(CLASS_MAP, {"classes": classes, "class_to_idx": class_to_idx})
    status(f"Saved class mapping: {CLASS_MAP}")

    # Build file list from balanced dataset
    files, labels = build_file_label_lists_from_folder(BALANCED_DIR, classes)
    labels_idx = np.array([class_to_idx[c] for c in labels], dtype=np.int32)

    # Split stratified
    status("Creating stratified splits...")
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        files, labels,
        test_size=CONFIG["test_split"],
        random_state=CONFIG["seed"],
        stratify=labels
    )
    val_relative = CONFIG["val_split"] / (1.0 - CONFIG["test_split"])
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval,
        test_size=val_relative,
        random_state=CONFIG["seed"],
        stratify=y_trainval
    )

    status(f"Split sizes: train={len(X_train)} val={len(X_val)} test={len(X_test)}")

    export_split("train", X_train, y_train, TRAIN_DIR)
    export_split("val", X_val, y_val, VAL_DIR)
    export_split("test", X_test, y_test, TEST_DIR)

    status("Creating TEST.zip...")
    zip_folder(TEST_DIR, TEST_ZIP)
    status(f"TEST.zip created: {TEST_ZIP} ({TEST_ZIP.stat().st_size/(1024*1024):.2f} MB)")

    # Build datasets from exported splits
    train_files, train_labels = build_file_label_lists_from_folder(TRAIN_DIR, classes)
    val_files, val_labels = build_file_label_lists_from_folder(VAL_DIR, classes)
    test_files, test_labels = build_file_label_lists_from_folder(TEST_DIR, classes)

    y_train_idx = np.array([class_to_idx[c] for c in train_labels], dtype=np.int32)
    y_val_idx = np.array([class_to_idx[c] for c in val_labels], dtype=np.int32)
    y_test_idx = np.array([class_to_idx[c] for c in test_labels], dtype=np.int32)

    train_ds = make_dataset(train_files, y_train_idx, len(classes), training=True)
    val_ds = make_dataset(val_files, y_val_idx, len(classes), training=False)
    test_ds = make_dataset(test_files, y_test_idx, len(classes), training=False)

    model, base = build_model(len(classes))
    compile_model(model, CONFIG["learning_rate"])

    callbacks = [
        keras.callbacks.ModelCheckpoint(
            filepath=str(BEST_WEIGHTS),
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=CONFIG["early_stop_patience"],
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=CONFIG["reduce_lr_patience"],
            min_lr=1e-7,
            verbose=1
        )
    ]

    status("Training phase 1 (frozen backbone)...")
    h1 = model.fit(train_ds, validation_data=val_ds, epochs=CONFIG["epochs"], callbacks=callbacks)

    # Fine-tune top layers (simple)
    status("Fine-tuning (unfreeze top 30%)...")
    base.trainable = True
    n = len(base.layers)
    cut = int(n * 0.7)
    for i, layer in enumerate(base.layers):
        layer.trainable = (i >= cut)

    compile_model(model, CONFIG["fine_tune_lr"])
    h2 = model.fit(train_ds, validation_data=val_ds, epochs=CONFIG["fine_tune_epochs"], callbacks=callbacks)

    history = {
        "accuracy": h1.history["accuracy"] + h2.history["accuracy"],
        "val_accuracy": h1.history["val_accuracy"] + h2.history["val_accuracy"],
        "loss": h1.history["loss"] + h2.history["loss"],
        "val_loss": h1.history["val_loss"] + h2.history["val_loss"],
    }

    status("Saving history CSV...")
    with open(HISTORY_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(history.keys())
        for i in range(len(history["loss"])):
            w.writerow([history[k][i] for k in history])

    status("Saving graphs...")
    plot_history(history, PLOTS_DIR)

    status("Evaluating on test set...")
    if BEST_WEIGHTS.exists():
        model.load_weights(str(BEST_WEIGHTS))

    y_prob = model.predict(test_ds, verbose=1)
    y_pred = np.argmax(y_prob, axis=1)

    cm = confusion_matrix(y_test_idx, y_pred)
    save_json(CONFUSION_JSON, {"confusion_matrix": cm.tolist()})
    plot_confusion_matrix(cm, classes, PLOTS_DIR / "confusion_matrix.png")
    plot_per_class_acc(cm, classes, PLOTS_DIR / "per_class_accuracy.png")

    rep = classification_report(y_test_idx, y_pred, target_names=classes, output_dict=True, zero_division=0)
    save_json(CLASS_REPORT_JSON, rep)

    status("Saving model...")
    model.save(str(MODEL_KERAS))
    model.save(str(MODEL_H5))
    if SAVEDMODEL_DIR.exists():
        shutil.rmtree(SAVEDMODEL_DIR, ignore_errors=True)
    model.export(str(SAVEDMODEL_DIR))  # Keras 3 safe

    status("✅ DONE.")
    print("Artifacts:")
    print(" -", RUN_DIR)
    print(" - Balanced dataset:", BALANCED_DIR)
    print(" - TEST.zip:", TEST_ZIP)
    print(" - Plots:", PLOTS_DIR)
    print(" - Reports:", REPORTS_DIR)
    print(" - Models:", MODELS_DIR)

if __name__ == "__main__":
    main()



🟦 GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

🟦 Counting images per class...
  - cardboard   : 1533
  - glass       : 1998
  - marine_trash: 7683
  - metal       : 993
  - oil_spill   : 768
  - paper       : 1380
  - plastic     : 1709
Total images: 16064

🟦 Saved counts: /kaggle/working/run_min_balance/reports/class_counts.json

🟦 Building balanced dataset (augment minority, delete only if necessary)...

🟦 Balance target per class: 1200


Copy metal: 100%|██████████| 993/993 [00:01<00:00, 577.28it/s]



🟦 Augmenting 'metal' by 207 to reach 1200...


Copy plastic: 100%|██████████| 1709/1709 [00:02<00:00, 581.69it/s]



🟦 Downsampling 'marine_trash': 7683 -> 3073 (deleted=4610)


...

Copying test: 100%|██████████| 1814/1814 [00:00<00:00, 6340.16it/s]



🟦 Creating TEST.zip...

🟦 TEST.zip created: /kaggle/working/run_min_balance/TEST.zip (18.66 MB)

🟦 Building MobileNetV2 (small + stable)...

🟦 Training phase 1 (frozen backbone)...
Epoch 1/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - accuracy: 0.7180 - loss: 1.0487
Epoch 1: val_loss improved from inf to 0.92775, saving model to /kaggle/working/run_min_balance/models/best.weights.h5
265/265 ━━━━━━━━━━━━━━━━━━━━ 132s 426ms/step - accuracy: 0.7181 - loss: 1.0485 - val_accuracy: 0.7233 - val_loss: 0.9277 - learning_rate: 3.0000e-04
Epoch 2/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.7052 - loss: 1.0054
Epoch 2: val_loss improved from 0.92775 to 0.78551, saving model to /kaggle/working/run_min_balance/models/best.weights.h5
265/265 ━━━━━━━━━━━━━━━━━━━━ 56s 207ms/step - accuracy: 0.7056 - loss: 1.0046 - val_accuracy: 0.7641 - val_loss: 0.7855 - learning_rate: 3.0000e-04
Epoch 3/20
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.7487 - loss: 0.9003
Epoch 3: val_lo

INFO:tensorflow:Assets written to: /kaggle/working/run_min_balance/models/saved_model/assets


INFO:tensorflow:Assets written to: /kaggle/working/run_min_balance/models/saved_model/assets


Saved artifact at '/kaggle/working/run_min_balance/models/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_518')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  132417950570576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950573072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950573456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950579600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950573264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950574608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950573840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950573648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950572496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417950575376: TensorSpec(shape=(), dtype=tf

In [9]:
import os
import zipfile
from pathlib import Path

RUN_DIR = Path("/kaggle/working/run_min_balance")
OUT_ZIP = RUN_DIR / "deploy_bundle.zip"

INCLUDE = [
    RUN_DIR / "models" / "model_best.keras",
    RUN_DIR / "models" / "class_mapping.json",
    RUN_DIR / "TEST.zip",
    RUN_DIR / "plots" / "confusion_matrix.png",
    RUN_DIR / "reports" / "classification_report.json",
    RUN_DIR / "reports" / "confusion_matrix.json",
    RUN_DIR / "models" / "saved_model",   # folder
]

def add_path(zf: zipfile.ZipFile, p: Path, arc_base: Path):
    if p.is_file():
        zf.write(p, arcname=str(p.relative_to(arc_base)))
    elif p.is_dir():
        for fp in p.rglob("*"):
            if fp.is_file():
                zf.write(fp, arcname=str(fp.relative_to(arc_base)))

if OUT_ZIP.exists():
    OUT_ZIP.unlink()

with zipfile.ZipFile(OUT_ZIP, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in INCLUDE:
        if not p.exists():
            print(f"⚠️ Missing, skipping: {p}")
            continue
        add_path(zf, p, RUN_DIR)

print("✅ Created:", OUT_ZIP)
print("📦 Size (MB):", round(OUT_ZIP.stat().st_size / (1024*1024), 2))
print("➡️ Download it from Kaggle output panel: /kaggle/working/run_min_balance/deploy_bundle.zip")


✅ Created: /kaggle/working/run_min_balance/deploy_bundle.zip
📦 Size (MB): 52.82
➡️ Download it from Kaggle output panel: /kaggle/working/run_min_balance/deploy_bundle.zip


In [10]:
import json, random
from pathlib import Path
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Paths
RUN_DIR = Path("/kaggle/working/run_min_balance")
MODEL_PATH = RUN_DIR / "models" / "model_best.keras"
MAP_PATH = RUN_DIR / "models" / "class_mapping.json"
TEST_DIR = RUN_DIR / "splits" / "test"

assert MODEL_PATH.exists(), f"Missing model: {MODEL_PATH}"
assert MAP_PATH.exists(), f"Missing class_mapping: {MAP_PATH}"
assert TEST_DIR.exists(), f"Missing test folder: {TEST_DIR}"

# Load mapping + model
mapping = json.loads(MAP_PATH.read_text())
classes = mapping["classes"]
class_to_idx = mapping["class_to_idx"]
idx_to_class = {v: k for k, v in class_to_idx.items()}

print("✅ Classes:", classes)

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded:", MODEL_PATH)

IMG_SIZE = 224

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

# Build test file list
test_files, test_labels = [], []
for cls in classes:
    folder = TEST_DIR / cls
    if not folder.exists():
        continue
    imgs = list_images(folder)
    test_files.extend([str(p) for p in imgs])
    test_labels.extend([cls] * len(imgs))

y_true = np.array([class_to_idx[c] for c in test_labels], dtype=np.int32)
print(f"✅ Test samples: {len(test_files)}")

# tf.data for test
def make_test_ds(paths):
    ds = tf.data.Dataset.from_tensor_slices(paths)

    def _load(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32)
        return img

    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
    return ds

test_ds = make_test_ds(test_files)

# Predict
probs = model.predict(test_ds, verbose=1)
y_pred = np.argmax(probs, axis=1)

acc = float(np.mean(y_pred == y_true))
print("\n✅ Overall Test Accuracy:", round(acc, 4))

# Confusion matrix + report
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=classes, zero_division=0))

# Per-class accuracy
print("\nPer-class accuracy:")
for i, cls in enumerate(classes):
    denom = cm[i].sum()
    per_acc = (cm[i, i] / denom) if denom > 0 else 0.0
    print(f"  - {cls:12s}: {per_acc:.3f} ({cm[i,i]}/{denom})")

# Manual random samples
print("\n🔎 Random sample predictions (from TEST split):")
sample_n = 12
idxs = random.sample(range(len(test_files)), min(sample_n, len(test_files)))

for k in idxs:
    path = test_files[k]
    true_cls = test_labels[k]
    p = probs[k]
    pred_idx = int(np.argmax(p))
    pred_cls = idx_to_class[pred_idx]
    conf = float(np.max(p))
    print(f"- True={true_cls:12s}  Pred={pred_cls:12s}  Conf={conf:.3f}  File={Path(path).name}")

print("\n✅ Done.")
print("If you want to test a specific image, paste its path and run the helper below.")


✅ Classes: ['cardboard', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']
✅ Model loaded: /kaggle/working/run_min_balance/models/model_best.keras
✅ Test samples: 1814
57/57 ━━━━━━━━━━━━━━━━━━━━ 23s 213ms/step

✅ Overall Test Accuracy: 0.9184

Confusion Matrix:
 [[190   1   0   9   0  28   2]
 [  0 252   0  18   1   0  29]
 [  0   0 460   0   1   0   0]
 [  1   0   0 172   0   0   7]
 [  0   0   0   1 179   0   0]
 [  4   4   0   9   2 180   8]
 [  3  10   0   5   0   5 233]]

Classification Report:

              precision    recall  f1-score   support

   cardboard       0.96      0.83      0.89       230
       glass       0.94      0.84      0.89       300
marine_trash       1.00      1.00      1.00       461
       metal       0.80      0.96      0.87       180
   oil_spill       0.98      0.99      0.99       180
       paper       0.85      0.87      0.86       207
     plastic       0.84      0.91      0.87       256

    accuracy                           0.92

In [12]:
from pathlib import Path

DATA_DIR = Path("/kaggle/input/processed-again-costal-polutant")

candidates = [
    DATA_DIR "/kaggle/input/procesed-again-costal-polutant/processed_marine_waste/clean_underwater",
    DATA_DIR / "processed_marine_waste" / "clean underwater",
    DATA_DIR / "processed_marine_waste" / "clean",
    DATA_DIR / "processed_oilspill" / "no_oil_spill",
    DATA_DIR / "processed_oilspill" / "no oil spill",
    DATA_DIR / "processed_oilspill" / "no_oil",
]

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

print("Checking candidates:")
for p in candidates:
    if p.exists():
        imgs = list_images(p)
        print(f"✅ {p}  -> {len(imgs)} images")
    else:
        print(f"❌ {p}  (missing)")


Checking candidates:
❌ /kaggle/input/processed-again-costal-polutant/processed_marine_waste/clean_underwater  (missing)
❌ /kaggle/input/processed-again-costal-polutant/processed_marine_waste/clean underwater  (missing)
❌ /kaggle/input/processed-again-costal-polutant/processed_marine_waste/clean  (missing)
❌ /kaggle/input/processed-again-costal-polutant/processed_oilspill/no_oil_spill  (missing)
❌ /kaggle/input/processed-again-costal-polutant/processed_oilspill/no oil spill  (missing)
❌ /kaggle/input/processed-again-costal-polutant/processed_oilspill/no_oil  (missing)


In [13]:
import json
import numpy as np
import tensorflow as tf
from pathlib import Path

RUN_DIR = Path("/kaggle/working/run_min_balance")
MODEL_PATH = RUN_DIR / "models" / "model_best.keras"
MAP_PATH = RUN_DIR / "models" / "class_mapping.json"

DATA_DIR = Path("/kaggle/input/processed-again-costal-polutant")

NEGATIVE_SOURCES = {
    "clean_underwater": DATA_DIR / "processed_marine_waste" / "clean_underwater",
    "no_oil_spill": DATA_DIR / "processed_oilspill" / "no_oil_spill",
}

mapping = json.loads(MAP_PATH.read_text())
classes = mapping["classes"]
idx_to_class = {v: k for k, v in mapping["class_to_idx"].items()}

model = tf.keras.models.load_model(MODEL_PATH)

IMG_SIZE = 224

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

neg_files = []
neg_names = []
for name, folder in NEGATIVE_SOURCES.items():
    if folder.exists():
        imgs = list_images(folder)
        neg_files.extend([str(x) for x in imgs])
        neg_names.extend([name]*len(imgs))

print("Negative samples:", len(neg_files))

if len(neg_files) == 0:
    print("🟨 No negative images found. Either the folders are missing/empty.")
    print("👉 Use a manual threshold like 0.60 for UNKNOWN gating, or add clean-water images.")
else:
    # Force dtype string explicitly
    paths = tf.constant(neg_files, dtype=tf.string)

    def _load(p):
        img = tf.io.read_file(p)
        # decode_image supports jpg/png safely
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32)
        return img

    ds = tf.data.Dataset.from_tensor_slices(paths).map(_load, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

    probs = model.predict(ds, verbose=1)
    maxp = np.max(probs, axis=1)
    pred = np.argmax(probs, axis=1)
    pred_cls = np.array([idx_to_class[int(i)] for i in pred])

    thresholds = [0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70]
    print("\nThreshold test (how many negatives still get flagged as a class):")
    for t in thresholds:
        flagged = np.mean(maxp >= t)
        print(f"  t={t:.2f} -> flagged negatives: {flagged*100:.2f}%")

    suggest = None
    for t in thresholds:
        if np.mean(maxp >= t) <= 0.05:
            suggest = t
            break
    print("\nSuggested threshold:", suggest)

    topk = np.argsort(-maxp)[:15]
    print("\nTop confident negative predictions (worst-case clean images):")
    for i in topk:
        print(f"  maxp={maxp[i]:.3f} pred={pred_cls[i]:12s} source={neg_names[i]} file={Path(neg_files[i]).name}")


Negative samples: 0
🟨 No negative images found. Either the folders are missing/empty.
👉 Use a manual threshold like 0.60 for UNKNOWN gating, or add clean-water images.


In [14]:
import json
import numpy as np
import tensorflow as tf
from pathlib import Path

RUN_DIR = Path("/kaggle/working/run_min_balance")
MODEL_PATH = RUN_DIR / "models" / "model_best.keras"
MAP_PATH = RUN_DIR / "models" / "class_mapping.json"

# ✅ Correct dataset root (your exact path)
DATA_DIR = Path("/kaggle/input/procesed-again-costal-polutant")

NEGATIVE_SOURCES = {
    "clean_underwater": DATA_DIR / "processed_marine_waste" / "clean_underwater",
    "no_oil_spill": DATA_DIR / "processed_oilspill" / "no_oil_spill",
}

mapping = json.loads(MAP_PATH.read_text())
idx_to_class = {v: k for k, v in mapping["class_to_idx"].items()}

model = tf.keras.models.load_model(MODEL_PATH)

IMG_SIZE = 224

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

# Collect negative files
neg_files = []
neg_names = []
for name, folder in NEGATIVE_SOURCES.items():
    print("Checking:", folder, "exists=", folder.exists())
    if folder.exists():
        imgs = list_images(folder)
        print(f"  -> found {len(imgs)} images")
        neg_files.extend([str(x) for x in imgs])
        neg_names.extend([name]*len(imgs))

print("\nNegative samples total:", len(neg_files))

if len(neg_files) == 0:
    print("🟨 Still zero negatives. Double-check the folder has images.")
else:
    # Force dtype string
    paths = tf.constant(neg_files, dtype=tf.string)

    def _load(p):
        img = tf.io.read_file(p)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32)
        return img

    ds = tf.data.Dataset.from_tensor_slices(paths)\
        .map(_load, num_parallel_calls=tf.data.AUTOTUNE)\
        .batch(32)\
        .prefetch(tf.data.AUTOTUNE)

    probs = model.predict(ds, verbose=1)
    maxp = np.max(probs, axis=1)
    pred = np.argmax(probs, axis=1)
    pred_cls = np.array([idx_to_class[int(i)] for i in pred])

    thresholds = [0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75]
    print("\nThreshold test (negatives flagged as some class):")
    for t in thresholds:
        flagged = np.mean(maxp >= t)
        print(f"  t={t:.2f} -> flagged negatives: {flagged*100:.2f}%")

    suggest = None
    for t in thresholds:
        if np.mean(maxp >= t) <= 0.05:
            suggest = t
            break
    print("\n✅ Suggested unknown threshold (<=5% false alarms):", suggest)

    topk = np.argsort(-maxp)[:15]
    print("\nTop confident NEGATIVE predictions (worst-case clean images):")
    for i in topk:
        print(f"  maxp={maxp[i]:.3f} pred={pred_cls[i]:12s} source={neg_names[i]} file={Path(neg_files[i]).name}")


Checking: /kaggle/input/procesed-again-costal-polutant/processed_marine_waste/clean_underwater exists= True
  -> found 1 images
Checking: /kaggle/input/procesed-again-costal-polutant/processed_oilspill/no_oil_spill exists= True
  -> found 412 images

Negative samples total: 413
13/13 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step

Threshold test (negatives flagged as some class):
  t=0.40 -> flagged negatives: 98.55%
  t=0.45 -> flagged negatives: 97.82%
  t=0.50 -> flagged negatives: 96.85%
  t=0.55 -> flagged negatives: 96.61%
  t=0.60 -> flagged negatives: 95.88%
  t=0.65 -> flagged negatives: 93.95%
  t=0.70 -> flagged negatives: 93.46%
  t=0.75 -> flagged negatives: 92.49%

✅ Suggested unknown threshold (<=5% false alarms): None

Top confident NEGATIVE predictions (worst-case clean images):
  maxp=1.000 pred=oil_spill    source=no_oil_spill file=no_oil_0000145.jpg
  maxp=1.000 pred=oil_spill    source=no_oil_spill file=no_oil_0000545.jpg
  maxp=1.000 pred=oil_spill    source=no_oil_spill file=n

In [15]:
import json, shutil, zipfile
from pathlib import Path
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# ===== Paths =====
RUN_DIR = Path("/kaggle/working/run_min_balance")
MODEL_PATH = RUN_DIR / "models" / "model_best.keras"
MAP_PATH = RUN_DIR / "models" / "class_mapping.json"

DATA_DIR = Path("/kaggle/input/procesed-again-costal-polutant")
NO_OIL_DIR = DATA_DIR / "processed_oilspill" / "no_oil_spill"

OUT_ROOT = Path("/kaggle/working/clean_water_filtered")
OUT_DIR = OUT_ROOT / "clean_water"
OUT_ZIP = Path("/kaggle/working/clean_water.zip")

# ===== Filters (tune these) =====
# 1) If model thinks it's oil_spill with high confidence -> reject as "not clean"
OIL_REJECT_CONF = 0.60

# 2) If model is very confident it's some trash class -> reject (not clean)
MAX_CONF_REJECT = 0.90

# 3) If model is uncertain overall -> accept as clean-ish
MAX_CONF_ACCEPT = 0.75

IMG_SIZE = 224

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def make_ds(paths):
    paths = tf.constant([str(p) for p in paths], dtype=tf.string)
    ds = tf.data.Dataset.from_tensor_slices(paths)

    def _load(p):
        img = tf.io.read_file(p)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32)
        return img, p

    return ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

# ===== Load model + mapping =====
mapping = json.loads(MAP_PATH.read_text())
classes = mapping["classes"]
class_to_idx = mapping["class_to_idx"]
idx_to_class = {v: k for k, v in class_to_idx.items()}
oil_idx = class_to_idx.get("oil_spill", None)

assert MODEL_PATH.exists(), f"Missing model: {MODEL_PATH}"
assert MAP_PATH.exists(), f"Missing mapping: {MAP_PATH}"
assert NO_OIL_DIR.exists(), f"Missing folder: {NO_OIL_DIR}"
assert oil_idx is not None, "oil_spill class not found in mapping!"

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Loaded model + mapping")

# ===== Collect files =====
files = list_images(NO_OIL_DIR)
print("✅ no_oil_spill images:", len(files))

if OUT_ROOT.exists():
    shutil.rmtree(OUT_ROOT, ignore_errors=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ===== Predict + filter =====
ds = make_ds(files)

kept = 0
rejected_oil = 0
rejected_confident = 0

all_paths = []
all_probs = []

# collect predictions
for batch_imgs, batch_paths in tqdm(ds, desc="Predicting"):
    probs = model.predict(batch_imgs, verbose=0)
    all_probs.append(probs)
    all_paths.extend(batch_paths.numpy().tolist())

all_probs = np.concatenate(all_probs, axis=0)

for i, pbytes in enumerate(all_paths):
    p = all_probs[i]
    maxp = float(np.max(p))
    pred_idx = int(np.argmax(p))
    pred_cls = idx_to_class[pred_idx]
    oilp = float(p[oil_idx])

    src_path = Path(pbytes.decode("utf-8"))

    # Reject if oil probability is high
    if oilp >= OIL_REJECT_CONF:
        rejected_oil += 1
        continue

    # Reject if model is extremely confident about any class (likely not clean)
    if maxp >= MAX_CONF_REJECT:
        rejected_confident += 1
        continue

    # Accept if model is overall uncertain (clean-ish)
    if maxp <= MAX_CONF_ACCEPT:
        dst = OUT_DIR / src_path.name
        shutil.copy2(src_path, dst)
        kept += 1

print("\n✅ Filtering done!")
print("Kept (clean-ish):", kept)
print("Rejected (oil-like):", rejected_oil)
print("Rejected (too confident):", rejected_confident)
print("Output folder:", OUT_DIR)

# ===== Zip for download =====
if OUT_ZIP.exists():
    OUT_ZIP.unlink()

with zipfile.ZipFile(OUT_ZIP, "w", zipfile.ZIP_DEFLATED) as zf:
    for fp in OUT_ROOT.rglob("*"):
        if fp.is_file():
            zf.write(fp, arcname=str(fp.relative_to(OUT_ROOT)))

print("\n📦 Created:", OUT_ZIP)
print("Size (MB):", round(OUT_ZIP.stat().st_size / (1024*1024), 2))
print("➡️ Download: /kaggle/working/clean_water.zip")


✅ Loaded model + mapping
✅ no_oil_spill images: 412


Predicting: 100%|██████████| 13/13 [00:36<00:00,  2.84s/it]


✅ Filtering done!
Kept (clean-ish): 27
Rejected (oil-like): 385
Rejected (too confident): 0
Output folder: /kaggle/working/clean_water_filtered/clean_water

📦 Created: /kaggle/working/clean_water.zip
Size (MB): 0.42
➡️ Download: /kaggle/working/clean_water.zip


In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnaud58/landscape-pictures")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/landscape-pictures


In [22]:
import shutil, zipfile
from pathlib import Path
from PIL import Image, ImageFile
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

# =========================
# CONFIG
# =========================
IMG_SIZE = 224
MAX_IMAGES = 2500  # cap so clean_water doesn't dominate; adjust if you want

LANDSCAPE_12K = Path("/kaggle/input/landscape-recognition-image-dataset-12k-images")

OUT_ROOT = Path("/kaggle/working/clean_water_dataset")
OUT_CLASS_DIR = OUT_ROOT / "clean_water"
OUT_ZIP = Path("/kaggle/working/clean_water.zip")

MERGE_TO_EXISTING = True
MERGE_TARGET = Path("/kaggle/working/run_min_balance/balanced_dataset")  # your working dataset

# =========================
# Helpers
# =========================
def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def preprocess_copy(img_paths, out_dir: Path, prefix: str):
    out_dir.mkdir(parents=True, exist_ok=True)
    kept = 0
    for fp in tqdm(img_paths, desc=f"Preprocess {prefix}"):
        try:
            im = Image.open(fp).convert("RGB")
            im = im.resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
            im.save(out_dir / f"{prefix}_{kept:07d}.jpg", quality=90, optimize=True)
            kept += 1
        except Exception:
            continue
    return kept

def make_zip(folder: Path, zip_path: Path):
    if zip_path.exists():
        zip_path.unlink()
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for fp in folder.rglob("*"):
            if fp.is_file():
                zf.write(fp, arcname=str(fp.relative_to(folder)))

# =========================
# Coast dirs (exact structure you showed)
# =========================
base = LANDSCAPE_12K / "Landscape Classification" / "Landscape Classification"

train_coast = base / "Training Data" / "Coast"
val_coast   = base / "Validation Data" / "Coast"
test_coast  = base / "Testing Data" / "Coast"

print("Coast train:", train_coast, "exists=", train_coast.exists())
print("Coast val  :", val_coast, "exists=", val_coast.exists())
print("Coast test :", test_coast, "exists=", test_coast.exists())

coast_imgs = []
for p in [train_coast, val_coast, test_coast]:
    coast_imgs += list_images(p)

print("\n✅ Total coast images found:", len(coast_imgs))

# cap
if MAX_IMAGES is not None and len(coast_imgs) > MAX_IMAGES:
    coast_imgs = coast_imgs[:MAX_IMAGES]
    print("🧢 Capped to:", len(coast_imgs))

# =========================
# Preprocess output
# =========================
if OUT_ROOT.exists():
    shutil.rmtree(OUT_ROOT, ignore_errors=True)
OUT_CLASS_DIR.mkdir(parents=True, exist_ok=True)

k = preprocess_copy(coast_imgs, OUT_CLASS_DIR, "coast")

print("\n✅ Saved clean_water images:", k)
print("Output folder:", OUT_CLASS_DIR)

# =========================
# Zip
# =========================
make_zip(OUT_ROOT, OUT_ZIP)
print("\n📦 Created zip:", OUT_ZIP)
print("➡️ Download: /kaggle/working/clean_water.zip")

# =========================
# Merge into dataset
# =========================
if MERGE_TO_EXISTING:
    target = MERGE_TARGET / "clean_water"
    target.mkdir(parents=True, exist_ok=True)
    for fp in OUT_CLASS_DIR.glob("*.jpg"):
        shutil.copy2(fp, target / fp.name)
    print("\n✅ Merged into:", target)
    print("Merged count:", len(list_images(target)))


Coast train: /kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Training Data/Coast exists= True
Coast val  : /kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Validation Data/Coast exists= True
Coast test : /kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Testing Data/Coast exists= True

✅ Total coast images found: 2400


Preprocess coast: 100%|██████████| 2400/2400 [00:11<00:00, 202.35it/s]



✅ Saved clean_water images: 2400
Output folder: /kaggle/working/clean_water_dataset/clean_water

📦 Created zip: /kaggle/working/clean_water.zip
➡️ Download: /kaggle/working/clean_water.zip

✅ Merged into: /kaggle/working/run_min_balance/balanced_dataset/clean_water
Merged count: 2400


In [24]:
import os, json, random, shutil, zipfile
from pathlib import Path
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# =========================
# CONFIG
# =========================
CONFIG = {
    "seed": 42,
    "img_size": 224,
    "batch_size": 32,
    "epochs_phase1": 10,
    "epochs_phase2": 15,
    "lr_phase1": 1e-3,
    "lr_phase2": 1e-5,
    "dropout": 0.4,
    "label_smoothing": 0.05,
    "unfreeze_top_percent": 0.30,
    "val_split": 0.15,
    "test_split": 0.15,

    # ✅ Safety cap: prevents clean_water or any class from dominating
    # Set to None to disable
    "max_per_class": 1200,
}

DATASET_DIR = Path("/kaggle/working/run_min_balance/balanced_dataset")
OUT_DIR = Path("/kaggle/working/run_final_train")
MODEL_DIR = OUT_DIR / "models"
PLOTS_DIR = OUT_DIR / "plots"
REPORTS_DIR = OUT_DIR / "reports"
SPLITS_DIR = OUT_DIR / "splits"
TEST_ZIP = OUT_DIR / "TEST.zip"

for d in [MODEL_DIR, PLOTS_DIR, REPORTS_DIR, SPLITS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

random.seed(CONFIG["seed"])
np.random.seed(CONFIG["seed"])
tf.random.set_seed(CONFIG["seed"])

# =========================
# GPU + mixed precision (safe)
# =========================
gpus = tf.config.list_physical_devices("GPU")
print("GPUs:", gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy("mixed_float16")
        print("✅ Mixed precision enabled")
    except Exception as e:
        print("⚠️ Mixed precision not enabled:", e)

# =========================
# Helpers
# =========================
def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def save_list(path, arr):
    with open(path, "w") as f:
        for x in arr:
            f.write(str(x) + "\n")

# =========================
# Load files (with cap safety)
# =========================
assert DATASET_DIR.exists(), f"Missing dataset dir: {DATASET_DIR}"

classes = sorted([p.name for p in DATASET_DIR.iterdir() if p.is_dir()])
print("✅ Classes found:", classes)

files, labels = [], []
raw_counts = {}

MAX_PER_CLASS = CONFIG["max_per_class"]

for c in classes:
    imgs = list_images(DATASET_DIR / c)
    raw_counts[c] = len(imgs)

    if MAX_PER_CLASS is not None and len(imgs) > MAX_PER_CLASS:
        # deterministic shuffle (seeded)
        rng = np.random.default_rng(CONFIG["seed"])
        idx = rng.permutation(len(imgs))[:MAX_PER_CLASS]
        imgs = [imgs[i] for i in idx]

    files.extend([str(p) for p in imgs])
    labels.extend([c] * len(imgs))

print("\n📊 Raw class counts (before cap):")
print(raw_counts)

final_counts = {c: labels.count(c) for c in classes}
print("\n📊 Final class counts (after cap):")
print(final_counts)

json.dump(raw_counts, open(REPORTS_DIR / "class_counts_raw.json", "w"), indent=2)
json.dump(final_counts, open(REPORTS_DIR / "class_counts_used.json", "w"), indent=2)

# mapping
class_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}

mapping = {"classes": classes, "class_to_idx": class_to_idx}
json.dump(mapping, open(MODEL_DIR / "class_mapping.json", "w"), indent=2)

X = np.array(files)
y = np.array([class_to_idx[c] for c in labels], dtype=np.int32)

# =========================
# Stratified split (70/15/15)
# =========================
tmp_split = CONFIG["val_split"] + CONFIG["test_split"]  # 0.30
X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y, test_size=tmp_split, stratify=y, random_state=CONFIG["seed"]
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=CONFIG["seed"]
)

print("\n✅ Split sizes:", len(X_train), len(X_val), len(X_test))

save_list(SPLITS_DIR/"train_files.txt", X_train)
save_list(SPLITS_DIR/"val_files.txt", X_val)
save_list(SPLITS_DIR/"test_files.txt", X_test)

# =========================
# Make TEST.zip (class folders)
# =========================
tmp_test_root = OUT_DIR / "test_folder_for_zip"
if tmp_test_root.exists():
    shutil.rmtree(tmp_test_root)
for c in classes:
    (tmp_test_root / c).mkdir(parents=True, exist_ok=True)

for fp, yi in zip(X_test, y_test):
    c = idx_to_class[int(yi)]
    shutil.copy2(fp, tmp_test_root / c / Path(fp).name)

if TEST_ZIP.exists():
    TEST_ZIP.unlink()
with zipfile.ZipFile(TEST_ZIP, "w", zipfile.ZIP_DEFLATED) as zf:
    for p in tmp_test_root.rglob("*"):
        if p.is_file():
            zf.write(p, arcname=str(p.relative_to(tmp_test_root)))

print("✅ TEST.zip created:", TEST_ZIP)

# =========================
# Data pipeline
# =========================
IMG_SIZE = (CONFIG["img_size"], CONFIG["img_size"])

augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
], name="augment")

def decode_resize(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    return img

def make_ds(paths, labels, training=False):
    paths = tf.constant(paths, dtype=tf.string)
    labels = tf.constant(labels, dtype=tf.int32)
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if training:
        ds = ds.shuffle(2048, seed=CONFIG["seed"], reshuffle_each_iteration=True)

    def _map(p, y):
        img = decode_resize(p)
        if training:
            img = augment(img, training=True)
        img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
        y_oh = tf.one_hot(y, depth=len(classes))
        return img, y_oh

    ds = ds.map(_map, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(CONFIG["batch_size"]).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(X_train, y_train, training=True)
val_ds = make_ds(X_val, y_val, training=False)
test_ds = make_ds(X_test, y_test, training=False)

# =========================
# Class weights
# =========================
cw = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights = {int(i): float(w) for i, w in zip(np.unique(y_train), cw)}
print("\n✅ Class weights:", class_weights)

# =========================
# Build small model (MobileNetV2)
# =========================
inputs = tf.keras.Input(shape=(CONFIG["img_size"], CONFIG["img_size"], 3))
base = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs)
base.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(CONFIG["dropout"])(x)

# keep output float32 for stability under mixed precision
outputs = tf.keras.layers.Dense(len(classes), activation="softmax", dtype="float32")(x)

model = tf.keras.Model(inputs, outputs)
print("\n✅ Model params:", model.count_params())

def compile_model(lr):
    opt = tf.keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=CONFIG["label_smoothing"])
    model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

ckpt_path = MODEL_DIR / "model_best.keras"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, verbose=1),
]

# =========================
# Train phase 1
# =========================
print("\n🟦 Phase 1 (frozen base)...")
compile_model(CONFIG["lr_phase1"])
h1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG["epochs_phase1"],
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# =========================
# Train phase 2
# =========================
print("\n🟦 Phase 2 (fine-tune top %)...")
base.trainable = True
n_layers = len(base.layers)
unfreeze_from = int(n_layers * (1 - CONFIG["unfreeze_top_percent"]))

for i, layer in enumerate(base.layers):
    layer.trainable = (i >= unfreeze_from)

compile_model(CONFIG["lr_phase2"])
h2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG["epochs_phase2"],
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# =========================
# Load best + evaluate
# =========================
best_model = tf.keras.models.load_model(ckpt_path)
print("\n✅ Loaded best:", ckpt_path)

probs = best_model.predict(test_ds, verbose=1)
y_pred = np.argmax(probs, axis=1)
y_true = y_test

acc = float(np.mean(y_pred == y_true))
print("\n✅ Test accuracy:", round(acc, 4))

cm = confusion_matrix(y_true, y_pred)
rep_text = classification_report(y_true, y_pred, target_names=classes, zero_division=0)
rep_dict = classification_report(y_true, y_pred, target_names=classes, zero_division=0, output_dict=True)

json.dump(cm.tolist(), open(REPORTS_DIR/"confusion_matrix.json","w"), indent=2)
json.dump(rep_dict, open(REPORTS_DIR/"classification_report.json","w"), indent=2)
print("\nClassification Report:\n", rep_text)

# =========================
# Plots
# =========================
def merged_history(histories, key):
    out = []
    for h in histories:
        out += h.history.get(key, [])
    return out

train_acc = merged_history([h1, h2], "accuracy")
val_acc = merged_history([h1, h2], "val_accuracy")
train_loss = merged_history([h1, h2], "loss")
val_loss = merged_history([h1, h2], "val_loss")

plt.figure()
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel("epoch"); plt.ylabel("accuracy")
plt.legend(["train","val"])
plt.title("Accuracy")
plt.savefig(PLOTS_DIR/"accuracy.png", bbox_inches="tight")
plt.close()

plt.figure()
plt.plot(train_loss)
plt.plot(val_loss)
plt.xlabel("epoch"); plt.ylabel("loss")
plt.legend(["train","val"])
plt.title("Loss")
plt.savefig(PLOTS_DIR/"loss.png", bbox_inches="tight")
plt.close()

plt.figure(figsize=(8,6))
plt.imshow(cm)
plt.xticks(range(len(classes)), classes, rotation=45, ha="right")
plt.yticks(range(len(classes)), classes)
plt.title("Confusion Matrix")
plt.colorbar()
plt.tight_layout()
plt.savefig(PLOTS_DIR/"confusion_matrix.png", bbox_inches="tight")
plt.close()

print("\n✅ Plots saved to:", PLOTS_DIR)

# =========================
# Save formats
# =========================
best_model.save(MODEL_DIR/"model_best.h5")               # optional legacy
best_model.export(str(MODEL_DIR/"saved_model"))         # SavedModel for serving/tflite
print("\n✅ Saved:")
print(" -", MODEL_DIR/"model_best.keras")
print(" -", MODEL_DIR/"model_best.h5")
print(" -", MODEL_DIR/"saved_model")
print(" -", MODEL_DIR/"class_mapping.json")
print(" -", TEST_ZIP)

print("\n📦 Outputs in:", OUT_DIR)


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
✅ Mixed precision enabled
✅ Classes found: ['cardboard', 'clean_water', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']

📊 Raw class counts (before cap):
{'cardboard': 1533, 'clean_water': 2400, 'glass': 1998, 'marine_trash': 3073, 'metal': 1200, 'oil_spill': 1200, 'paper': 1380, 'plastic': 1709}

📊 Final class counts (after cap):
{'cardboard': 1200, 'clean_water': 1200, 'glass': 1200, 'marine_trash': 1200, 'metal': 1200, 'oil_spill': 1200, 'paper': 1200, 'plastic': 1200}

✅ Split sizes: 6720 1440 1440
✅ TEST.zip created: /kaggle/working/run_final_train/TEST.zip

✅ Class weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0}


/tmp/ipykernel_55/2281044373.py:213: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

✅ Model params: 2273352

🟦 Phase 1 (frozen base)...
Epoch 1/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.5158 - loss: 1.6153
Epoch 1: val_loss improved from inf to 0.71316, saving model to /kaggle/working/run_final_train/models/model_best.keras
210/210 ━━━━━━━━━━━━━━━━━━━━ 82s 266ms/step - accuracy: 0.5165 - loss: 1.6133 - val_accuracy: 0.8271 - val_loss: 0.7132 - learning_rate: 0.0010
Epoch 2/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.7838 - loss: 0.8784
Epoch 2: val_loss improved from 0.71316 to 0.66513, saving model to /kaggle/working/run_final_train/models/model_best.keras
210/210 ━━━━━━━━━━━━━━━━━━━━ 46s 217ms/step - accuracy: 0.7838 - loss: 0.8783 - val_accuracy: 0.8590 - val_loss: 0.6651 - learning_rate: 0.0010
Epoch 3/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.8031 - loss: 0.8194
Epoch 3: val_loss improved from 0.66513 to 0.65454, saving model to /kaggle/working/run_final_train/mo


✅ Plots saved to: /kaggle/working/run_final_train/plots
INFO:tensorflow:Assets written to: /kaggle/working/run_final_train/models/saved_model/assets


INFO:tensorflow:Assets written to: /kaggle/working/run_final_train/models/saved_model/assets


Saved artifact at '/kaggle/working/run_final_train/models/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_10')]
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  132417294254160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417268559568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417268558992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417294254544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417294258960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417268563024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417268559184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132417268559376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132420779100304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132420779114320: TensorSpec(shape=(), dtyp

In [27]:
import shutil, zipfile
from pathlib import Path

RUN_DIR = Path("/kaggle/working/run_final_train")

MODEL_DIR   = RUN_DIR / "models"
PLOTS_DIR   = RUN_DIR / "plots"
REPORTS_DIR = RUN_DIR / "reports"
TEST_ZIP    = RUN_DIR / "TEST.zip"

BEST_KERAS  = MODEL_DIR / "model_best.keras"
CLASS_MAP   = MODEL_DIR / "class_mapping.json"
SAVED_MODEL = MODEL_DIR / "saved_model"

assert BEST_KERAS.exists(), f"Missing model: {BEST_KERAS}"
assert CLASS_MAP.exists(), f"Missing mapping: {CLASS_MAP}"

# Output bundle folder + zip path
BUNDLE_DIR = Path("/kaggle/working/deploy_bundle")
DEPLOY_ZIP = Path("/kaggle/working/deploy_bundle.zip")

# Reset bundle folder
if BUNDLE_DIR.exists():
    shutil.rmtree(BUNDLE_DIR, ignore_errors=True)
BUNDLE_DIR.mkdir(parents=True, exist_ok=True)

# 1) Copy model files
(BUNDLE_DIR / "models").mkdir(exist_ok=True)
shutil.copy2(BEST_KERAS, BUNDLE_DIR / "models" / BEST_KERAS.name)
shutil.copy2(CLASS_MAP,  BUNDLE_DIR / "models" / CLASS_MAP.name)

# Copy saved_model if present
if SAVED_MODEL.exists() and SAVED_MODEL.is_dir():
    shutil.copytree(SAVED_MODEL, BUNDLE_DIR / "models" / "saved_model", dirs_exist_ok=True)

# 2) Copy optional artifacts
if REPORTS_DIR.exists() and REPORTS_DIR.is_dir():
    shutil.copytree(REPORTS_DIR, BUNDLE_DIR / "reports", dirs_exist_ok=True)

if PLOTS_DIR.exists() and PLOTS_DIR.is_dir():
    shutil.copytree(PLOTS_DIR, BUNDLE_DIR / "plots", dirs_exist_ok=True)

if TEST_ZIP.exists():
    shutil.copy2(TEST_ZIP, BUNDLE_DIR / TEST_ZIP.name)

# 3) Add README
(BUNDLE_DIR / "README.txt").write_text(
"""Deploy bundle contents:

models/
  - model_best.keras
  - class_mapping.json
  - saved_model/ (optional)

reports/ (optional)
plots/   (optional)
TEST.zip (optional)

Backend rules suggestion:
- If predicted == clean_water -> no pollution detected
- Optional: if max_prob < 0.60 -> unknown
"""
)

# 4) Zip it
if DEPLOY_ZIP.exists():
    DEPLOY_ZIP.unlink()

with zipfile.ZipFile(DEPLOY_ZIP, "w", zipfile.ZIP_DEFLATED) as zf:
    for fp in BUNDLE_DIR.rglob("*"):
        if fp.is_file():
            zf.write(fp, arcname=str(fp.relative_to(BUNDLE_DIR)))

print("✅ Bundle folder:", BUNDLE_DIR)
print("📦 Zip created:", DEPLOY_ZIP)
print("Size (MB):", round(DEPLOY_ZIP.stat().st_size / (1024*1024), 2))
print("➡️ Download from Kaggle output: /kaggle/working/deploy_bundle.zip")


✅ Bundle folder: /kaggle/working/deploy_bundle
📦 Zip created: /kaggle/working/deploy_bundle.zip
Size (MB): 65.47
➡️ Download from Kaggle output: /kaggle/working/deploy_bundle.zip


In [28]:
import os, zipfile
from pathlib import Path
from tqdm import tqdm

DATA_DIR = Path("/kaggle/working/run_min_balance/balanced_dataset")
OUT_ZIP = Path("/kaggle/working/balanced_dataset.zip")

assert DATA_DIR.exists(), f"Dataset folder not found: {DATA_DIR}"

def list_images(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

# 1) Print counts
classes = sorted([p for p in DATA_DIR.iterdir() if p.is_dir()])
print("✅ Classes:", [c.name for c in classes])

total = 0
print("\n📊 Images per class:")
for c in classes:
    n = len(list_images(c))
    total += n
    print(f" - {c.name:15s}: {n}")

print("\n✅ Total images:", total)

# 2) Zip it
if OUT_ZIP.exists():
    OUT_ZIP.unlink()

all_files = [p for p in DATA_DIR.rglob("*") if p.is_file()]
print("\n🟦 Zipping... files:", len(all_files))

with zipfile.ZipFile(OUT_ZIP, "w", zipfile.ZIP_DEFLATED) as zf:
    for fp in tqdm(all_files):
        zf.write(fp, arcname=str(fp.relative_to(DATA_DIR)))

print("\n📦 Done!")
print("Zip path:", OUT_ZIP)
print("Size (GB):", round(OUT_ZIP.stat().st_size / (1024**3), 3))
print("➡️ Download from Kaggle output: /kaggle/working/balanced_dataset.zip")


✅ Classes: ['cardboard', 'clean_water', 'glass', 'marine_trash', 'metal', 'oil_spill', 'paper', 'plastic']

📊 Images per class:
 - cardboard      : 1533
 - clean_water    : 2400
 - glass          : 1998
 - marine_trash   : 3073
 - metal          : 1200
 - oil_spill      : 1200
 - paper          : 1380
 - plastic        : 1709

✅ Total images: 14493

🟦 Zipping... files: 14493


100%|██████████| 14493/14493 [00:06<00:00, 2195.83it/s]


📦 Done!
Zip path: /kaggle/working/balanced_dataset.zip
Size (GB): 0.155
➡️ Download from Kaggle output: /kaggle/working/balanced_dataset.zip
